# This is a demo to implement step-by-step AWmeta analysis

### 1️⃣ Make sure you have opened AWmeta folder:

In [ ]:
getwd()

[1] "/root/AWmeta"

### 2️⃣ Raw gene expression and corresponding clinical/phenotype files, should be stored in separate folders:
- Raw expression matrix files in [./data/raw_expr](https://github.com/YanshiHu/AWmeta/tree/main/data/raw_expr).
- Corresponding clinical/phenotype files in [./data/raw_clin](https://github.com/YanshiHu/AWmeta/tree/main/data/raw_clin).

In [8]:
# Raw expression matrix files should be stored in a directory:

print(list.files("/root/AWmeta/data/raw_expr"), quote = FALSE, sep = "\n")

[1] GSE165082.csv GSE18838.csv  GSE34287.csv  GSE54536.csv  GSE57475.csv 
[6] GSE6613.csv   GSE72267.csv  GSE99039.csv 


In [9]:
# Corresponding clinical/phenotype files should be stored in a directory:

print(list.files("/root/AWmeta/data/raw_clin"), quote = FALSE, sep = "\n")

[1] GSE165082_pheno.csv GSE18838_pheno.csv  GSE34287_pheno.csv 
[4] GSE54536_pheno.csv  GSE57475_pheno.csv  GSE6613_pheno.csv  
[7] GSE72267_pheno.csv  GSE99039_pheno.csv 


### 3️⃣ Raw gene expression data files, should be formatted as follows:
- `Row Name`:   
    - gene identifier, e.g., Entrez Gene ID (used here), Ensembl gene ID, gene symbol.
- `Col Name`:
    - sample identifier.
- `Separator`:
    - comma (",") used here.

⚠️ TO BE NOTED:
-  Microarray datasets should be processed to normalized intensity signals by limma, affy, or other relevant packages.
-  RNA-seq datasets should contain gene count matrices, quantified by featureCounts, STAR, or other related tools.

In [15]:
# Take GSE18838 as example:

head(read.table("data/raw_expr/GSE18838.csv", sep = ",", header = TRUE, row.names = 1, check.names = FALSE))

,GSM466881,GSM466882,GSM466883,GSM466884,GSM466885,GSM466886,GSM466887,GSM466888,GSM466889,GSM466890,⋯,GSM466899,GSM466900,GSM466901,GSM466902,GSM466903,GSM466904,GSM466905,GSM466906,GSM466907,GSM466908
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5.974322,5.529198,5.905892,5.869029,5.450818,5.718872,5.972054,5.574497,5.248609,6.056391,⋯,5.762262,6.146706,5.674313,5.849043,5.326472,5.713848,5.585179,5.572774,6.004133,5.933502
10,1.766114,1.478747,1.818198,1.889750,1.549316,1.932048,1.942990,2.261588,1.634396,2.521888,⋯,2.064368,3.498965,1.904373,1.877441,1.815903,1.847098,2.169656,1.537277,1.919092,2.457791
100,6.340801,6.246526,6.246568,6.500481,6.035279,6.276709,5.721718,5.947306,6.225666,6.166099,⋯,6.168828,6.141265,6.081029,6.212898,6.266483,6.294748,6.297601,6.207059,6.594319,6.400845
1000,3.279626,3.066588,3.461183,3.164990,3.017854,3.131056,3.603721,3.109295,3.164006,3.897606,⋯,3.282294,3.653656,3.277332,2.978603,3.848394,2.937089,3.739835,3.380353,3.449250,3.105334
10000,4.696992,4.467682,3.750259,4.430248,4.604152,4.366680,4.818588,4.377542,4.968625,3.752643,⋯,4.687619,4.206344,4.889710,4.497731,4.431595,4.070111,4.120799,4.165406,3.863490,4.052750
100008586,2.582434,2.682665,2.990375,2.120916,2.122672,2.633500,2.920980,2.014738,2.598039,3.368453,⋯,3.730438,2.679535,2.463807,1.928015,1.979664,2.896218,2.551836,2.482334,2.784737,3.352059


### 4️⃣ Corresponding clinical/phenotype files, should be formatted as follows:
- `Row Name`:   
    - same sample identifiers as raw gene expression data file.
- `Col Name`:  
    - "label"
- `Value`:     
    - case / control labels for corresponding samples ("control" and "PD" used here).

In [14]:
# Take GSE18838 as example:

read.table("data/raw_clin/GSE18838_pheno.csv", sep = ",", header = TRUE, row.names = 1, check.names = FALSE)

,label
,<chr>
GSM466881,PD
GSM466882,PD
GSM466883,PD
GSM466884,PD
GSM466885,PD
GSM466886,PD
GSM466887,PD
GSM466888,PD
GSM466889,PD


### 5️⃣ Implement AWmeta for adaptively-weighted transcriptomic meta-analysis:

In [50]:
# Load AWmeta method:

source("AWmeta.R")

In [51]:
# Run AWmeta to implement adaptively-weighted transcriptomic meta-analysis:

res <- AWmeta(
  raw.data.dir  = "./data/raw_expr",           # A path to the raw expression data files
  raw.clin.dir  = "./data/raw_clin",           # A path to the clinical/phenotype data files
  raw.sep       = ",",                         # Field separator used in raw expression and clinical/phenotype data files
  DE.method     = c("DESeq2", "limma"),        # DE method(s)
  compare.group = c("control", "PD"),          # Compared groups for DE analysis
  ref.level     = "control",                   # Compared reference for DE analysis
  paired        = FALSE,                       # Whether two-class samples are paired
  core.num      = 30                           # Number of CPU cores to be used
)

Loading data...

Setting up DE analysis parameters...

Auto-detecting data types to assign DE methods:

  - Study 'GSE165082': Detected discrete data.

  - Study 'GSE18838': Detected continuous data.

  - Study 'GSE34287': Detected continuous data.

  - Study 'GSE54536': Detected continuous data.

  - Study 'GSE57475': Detected continuous data.

  - Study 'GSE6613': Detected continuous data.

  - Study 'GSE72267': Detected continuous data.

  - Study 'GSE99039': Detected continuous data.

Performing differential expression analysis for each study...

  - Analyzing study: GSE165082

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 92 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



dataset 1 is done


  - Analyzing study: GSE18838



dataset 1 is done


  - Analyzing study: GSE34287



...

  - Analyzing study: GSE72267



dataset 1 is done


  - Analyzing study: GSE99039



dataset 1 is done


Formatting p-values for meta-analysis...

Running AW-Fisher to combine p-values...

Calculating FDR...

Preparing data for AWmeta fold-change calculation...

Running AW-REM to calculate AWmeta fold-change using 30 CPU cores...

Sorting results by P-value...

Meta-analysis complete.



### 6️⃣ Showcase AWmeta result:
- `Row Name`:   
    - gene identifier, e.g., Entrez Gene ID (used here), Ensembl gene ID, gene symbol.
- `Col Name`:  
    - per-study fold-changes, _P_-values, and AW weights.
    - AWmeta-derived _P_-values, FDRs and fold-changes.

In [52]:
head(res)

,GSE165082_FC,GSE18838_FC,GSE34287_FC,GSE54536_FC,GSE57475_FC,GSE6613_FC,GSE72267_FC,GSE99039_FC,GSE165082_Pvalue,GSE18838_Pvalue,⋯,GSE18838_AWmeta_Weight,GSE34287_AWmeta_Weight,GSE54536_AWmeta_Weight,GSE57475_AWmeta_Weight,GSE6613_AWmeta_Weight,GSE72267_AWmeta_Weight,GSE99039_AWmeta_Weight,AWmeta_P_value,AWmeta_FDR,AWmeta_FC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5730,0.16978281,-0.2657432,0.7763220,0.05504498,0.305381866,-0.1638967,0.3802264,0.4190457,0.6008361,0.070003486,⋯,0,0,0,1,0,0,1,8.835110e-09,0.0001072630,0.39957567
6404,-0.16005029,-0.8533483,0.1094780,0.47841000,0.522559318,-0.3024088,0.0140958,0.1847864,0.3368307,0.002511431,⋯,1,0,1,1,0,0,1,8.937094e-09,0.0001072630,0.11170200
3107,-0.07907529,-0.3542387,0.7776831,1.87841569,0.463393489,0.1237813,0.2048841,0.1694564,0.6539423,0.046168516,⋯,1,0,1,1,0,1,1,2.584098e-08,0.0002067623,0.40593148
222487,-0.19547606,-0.4686609,0.7988970,0.29980032,0.264581027,0.3074672,0.3574406,0.3886088,0.2800083,0.111064401,⋯,1,0,1,1,1,1,1,1.000800e-07,0.0005266034,0.31473664
3804,0.17793672,-0.7742791,-0.2087129,0.69469457,0.375824507,-0.1098050,0.1500876,0.2493906,0.7067727,0.023042608,⋯,1,0,0,1,0,0,1,1.096908e-07,0.0005266034,0.01174222
139189,NA,NA,0.1137890,-0.02825487,0.002139725,NA,NA,0.2180996,NA,NA,⋯,NA,0,0,0,NA,NA,1,1.399690e-07,0.0005599694,0.21809959
